**Measuring Demand Forecasting benefits series**

# Measuring forecast benefits

> *This notebook should work with the **`Data Science 3.0`** kernel in SageMaker Studio, and the default `ml.t3.medium` instance type (2 vCPU + 4 GiB RAM)*

In this notebook, we'll analyze the performance of the baseline (moving average) and ML-powered (Amazon SageMaker Canvas and/or Amazon Forecast) forecasts against actual historical data, and go beyond raw accuracy metrics to estimate actual business value.

⚠️ If you haven't already prepared your baseline and ML-powered forecast, go back to run the [1.1. Moving Average Baseline.ipynb](1.1.%20Moving%20Average%20Baseline.ipynb) and either [1.2. Run SageMaker Canvas.ipynb](1.2.%20Run%SageMaker%20Canvas.ipynb) or [1.3. Run Amazon Forecast.ipynb](1.3.%20Run%20Amazon%20Forecast.ipynb) first.

## Contents

1. [Dependencies and setup](#setup)
1. [Load input data](#data)
    1. [Actual sales/demand](#actuals)
    1. [Forecast predictions](#predictions)
        1. [Moving average baseline forecast](#movavg)
        1. [(Optional) SageMaker Canvas predictions](#canvas)
        1. [(Optional) Amazon Forecast predictions](#amzforecast)
1. [Estimating downstream costs of forecasting errors](#costs)
    1. [Cost of excess inventory](#inventory)
    1. [Cost of stock-out events](#stockouts)
    1. [Total costs and benefits](#totalcosts)
1. [Online evaluation and A/B testing](#abtesting)
1. [Conclusions](#conclusions)

## Dependencies and setup<a class="anchor" id="setup"></a>

As before we'll first load libraries needed by the rest of the notebook:

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
from dataclasses import dataclass
from datetime import datetime
from typing import Dict, List, Optional

# External Dependencies:
import pandas as pd  # Tabular/dataframe processing tools
import sagemaker  # SageMaker SDK used just to look up default S3 bucket

# Local Dependencies:
import util

## Load actuals and forecasts<a class="anchor" id="data"></a>

To retrospectively evaluate demand forecast(s) in a given period, we'll need:

- **The actual observed demand/sales** from the period
- **The predictions** of the forecast models - assuming multiple models with the goal to compare between them
- **Additional reference data** - to relate demand estimate errors to actual business costs

The following sub-sections will load and normalize the forecasts to be reviewed and the actual observed sales for the period, and discuss our assumptions on their structure.

First, we'll define the evaluation period and what information will be collected for each forecast:

In [ ]:
PERIOD_START = datetime(year=2019, month=12, day=1)
PERIOD_END = datetime(year=2020, month=1, day=1)  # (Exclusive)


@dataclass
class NormalizedForecast:
    name: str  # Some kind of human-ready identifier
    df: pd.DataFrame  # Filtered to the eval period, and indexed the same as the actuals data
    quantiles: List[str]  # (Multiple) alternative forecasts/quantile columns from this model


FORECASTS: Dict[str, NormalizedForecast] = {}

### Load actual sales/demand<a class="anchor" id="actuals"></a>

Now that the original forecast period is over, you should have actual sales data to compare your forecast to.

In [ ]:
actuals_timestamp_col = "date"
actuals_amt_col = "sales"

actuals_df = pd.read_parquet(
    "s3://measuring-forecast-benefits-assets/dataset/v1/sales.parquet",
).rename(columns={"sku": "item_id"})

actuals_dimensions = [
    col for col in actuals_df if col not in (actuals_timestamp_col, actuals_amt_col)
]
print(f"Data has breakdown dimensions: {actuals_dimensions}")

actuals_df

To anchor the comparison between forecasts (which may have different gaps from different models), we'll want a standard list of what dimension combinations (item-locations) to consider.

We'll take that from the sales actuals here... But remember that sales can be sparse due to low-volume items: An item might have had a forecast for the period, but never sold any units. So we'll extract the list **before filtering** by the analysis period start/end:

In [ ]:
# Calculate all unique location-item_id combinations from actual data:
loc_item_combos = actuals_df[["location", "item_id"]].drop_duplicates().reset_index(drop=True)

# Split the country and product out and index by these columns (helpful for joins later):
loc_item_combos["country"] = loc_item_combos["location"].str.split("_").str[0]
loc_item_combos["product"] = loc_item_combos["item_id"].str.split("_").str[0]

loc_item_combos

In this example the source (sales) data is daily but the baseline (rolling average) forecast is monthly only - so we'll need to conduct our comparative analysis at the monthly level. This is realistic as many businesses make stocking decisions at a similar frequency.

So after filtering to our period of interest, we'll also need to aggregate the actual demand up to a monthly basis:

In [ ]:
# Filter to just the analysis period:
actuals_df = util.analytics.filter_to_period(
    actuals_df,
    period_start=PERIOD_START,
    period_end=PERIOD_END,
    timestamp_col_name=actuals_timestamp_col,
)

# Ensure the source timestamp data is properly parsed to datetimes:
actuals_df[actuals_timestamp_col] = pd.to_datetime(actuals_df[actuals_timestamp_col])
# Create the month column:
actuals_df["month"] = actuals_df[actuals_timestamp_col].dt.strftime("%Y-%m")

# Aggregate the data (sum):
actuals_df = actuals_df.groupby(
    ["month"] + actuals_dimensions
).agg(
    {actuals_amt_col: "sum"}
)

# Going forward, the timestamp column for this DF is updated:
actuals_timestamp_col = "month"

# Preview resulting dataframe:
actuals_df

Any forecasts we load for comparison must be normalized to use this same multi-level index of month/period and other dimensions (here item_id, location).

### Forecast predictions<a class="anchor" id="predictions"></a>

You'll likely have two or more candidate forecasts to compare, since it doesn't really make sense to discuss the "value" of one forecast by itself without some kind of business baseline to measure against.

In this example, we'll use the moving average baseline forecast, and compare it against the either the SageMaker Canvas or Amazon Forecast model (whichever you created).

#### Moving average baseline forecast<a class="anchor" id="movavg"></a>

The moving average baseline forecast was calculated in our first notebook and saved locally:

In [ ]:
mov_avg_df = util.analytics.filter_to_period(
    pd.read_csv("dataset/moving_avg.csv"),
    period_start=PERIOD_START,
    period_end=PERIOD_END,
    timestamp_col_name="month",
)

mov_avg_df

This forecast is already stored in a monthly format so there's no aggregation to do in this case.

However, we'd like to:
- Map column names to the standard set as used in actuals data, and
- Ensure the records are *indexed* by the unique fields they should be, ready for data joins

Below we'll perform that renaming and re-indexing, and check that no records are merged in the process:

In [ ]:
mov_avg_len_prev = len(mov_avg_df)

# Re-name columns:
mov_avg_df.rename(
    columns={
        "sku": "item_id",  # Needs to match actuals data
        "mov_avg": "movavg",  # We'd like to avoid underscores in quantile names later
    },
    inplace=True,
)

# Index by month and item/dimensions:
mov_avg_df = mov_avg_df.groupby(["month"] + actuals_dimensions).agg({"movavg": "sum"})

# Check record count was not changed by the re-indexing / "aggregation":
assert len(mov_avg_df) == mov_avg_len_prev, (
    "Moving average forecast data changed length during re-indexing! Did you have duplicated "
    "records or incorrect dimension settings? (From %s to %s records)"
    % (mov_avg_len_prev, len(mov_avg_df))
)

mov_avg_df

This normalized data is now ready for comparison, so we'll add it to our list:

In [ ]:
FORECASTS["Moving Average"] = NormalizedForecast(
    name="Moving Average",
    df=mov_avg_df,
    quantiles=["movavg"],
)

#### (Optional) SageMaker Canvas predictions<a class="anchor" id="canvas"></a>

IF you ran through the [SageMaker Canvas notebook](1.2.%20Run%20SageMaker%20Canvas.ipynb), you should have a **donwloaded CSV file** of predictions from the model.

▶️ **Open** the `dataset` folder here in SageMaker Studio, using the folder menu in the left sidebar.

▶️ **Drag and drop** your Canvas result file from your computer to the folder area, to upload it to your SageMaker workspace. (⏰ This might take a few minutes to complete - see the upload progress bar at the bottom of the screen for current status)

▶️ **Check** the file location in the code cell below and edit it to match your uploaded file.

In [ ]:
canvas_result_df = util.analytics.filter_to_period(
    pd.read_csv("dataset/canvas_result.csv"),  # TODO: EDIT YOUR FILENAME/PATH AS NEEDED
    period_start=PERIOD_START,
    period_end=PERIOD_END,
    timestamp_col_name="date",
)

# Check missing values:
missing_by_field = canvas_result_df.isna().sum()
print(f"\nTotal missing values:\n{missing_by_field}")
if missing_by_field.sum() > 0:
    raise ValueError(
        "There are missing values in your SageMaker Canvas prediction result, which is most likely "
        "caused by an error in download/upload. Please retry uploading your forecasts to Studio - "
        "and possibly re-downloading them from Canvas if the error persists."
    )

# Preview the data:
canvas_result_df

There are a few transformations we need to apply to this dataset ready for comparison with the moving average baseline:

1. Since the model model has daily granularity data in this case, we'll need to aggregate the results to monthly for comparable metrics.
1. Canvas has lower-cased our `sku` and `location` values which would interfere with joins later. We can use the original values from `loc_item_combos` collected earlier to fix this.
1. As business logic, we'll also enforce that any predictions that turn out negative for the month as a whole are set to zero.
1. We'll need to ensure the dimension column names match and the data is indexed by them, ready for joining to actuals.

The cell below combines these steps:

In [ ]:
# Fix locations back to original casing:
# Define the list of locations with lowercase equivalents
tmp_locs_lower = pd.DataFrame(
    {"loc": loc_item_combos["location"]}
).drop_duplicates().reset_index(drop=True)
tmp_locs_lower["loc_lower"] = tmp_locs_lower["loc"].str.lower()
# Join on to the dataframe and remove the old/temporary columns:
canvas_result_df = canvas_result_df.merge(
    tmp_locs_lower,
    left_on="location",
    right_on="loc_lower",
    how="left",
).drop(columns=["location", "loc_lower"]).rename(columns={"loc": "location"})
del tmp_locs_lower

# Fix product SKUs back to original casing:
# Define the list of SKUs with lowercase equivalents
tmp_skus_lower = pd.DataFrame(
    {"item_id": loc_item_combos["item_id"]}
).drop_duplicates().reset_index(drop=True)
tmp_skus_lower["item_id_lower"] = tmp_skus_lower["item_id"].str.lower()
# Join on to the dataframe and remove the old/temporary columns:
canvas_result_df = canvas_result_df.merge(
    tmp_skus_lower,
    left_on="sku",
    right_on="item_id_lower",
    how="left",
).drop(columns=["sku", "item_id_lower"])
del tmp_skus_lower

# Normalize and parse the timestamp column:
canvas_result_df.rename(columns={"date": "timestamp"}, inplace=True)
canvas_result_df["timestamp"] = pd.to_datetime(canvas_result_df["timestamp"])

# Create the month column:
canvas_result_df["month"] = canvas_result_df["timestamp"].dt.strftime("%Y-%m")

# Aggregate the data (sum):
canvas_result_df = canvas_result_df.groupby(
    ["month"] + actuals_dimensions
).agg(
    {
        "p10": "sum",
        "p50": "sum",
        "p90": "sum",
    }
)

# Force any negative (month-aggregated) predictions up to 0 sales:
canvas_result_df[canvas_result_df < 0] = 0

# Preview resulting dataframe:
canvas_result_df

This forecast now matches our standard format, so we can add it to the evaluation set:

In [ ]:
FORECASTS["SageMaker Canvas"] = NormalizedForecast(
    name="SageMaker Canvas",
    df=canvas_result_df,
    quantiles=["p10", "p50", "p90"],
)

#### Amazon Forecast predictions<a class="anchor" id="amzforecast"></a>

IF you ran through the [Amazon Forecast notebook](1.3.%20Run%20Amazon%20Forecast.ipynb), you should now have an **exported predictor backtest** and *optionally* also a forward-looking forecast.

Because of the way the date cut-offs in this example have been set up, it's the backtest export you'll need to use for comparison.

▶️ **Find** your exported `backtest_s3_uri` from the Amazon Forecast notebook or the Amazon Forecast Console, and fill it in below.

Backtest exports contain **two** datasets: The actual forecasted values, and the calculated accuracy metrics. For the `forecast_export_s3uri` below, we'll need to append `/forecasted-values` to your main backtest export URI, to select only the actual forecast folder:

In [ ]:
forecast_export_s3uri = (
    "s3://TODO - YOUR backtest export S3 URI from previous notebook"
    + "/forecasted-values"
)

amz_forecast_df = util.analytics.filter_to_period(
    pd.read_parquet(forecast_export_s3uri),
    period_start=PERIOD_START,
    period_end=PERIOD_END,
    # timestamp_col_name="date",  # backtest uses 'timestamp' already, forecast would use 'date'
)
amz_forecast_df

Since we trained a model with daily granularity data in this case, we'll need to aggregate the results to monthly for comparable metrics.

As business logic, we'll also enforce that any predictions that turn out negative for the month as a whole are set to zero.

In [ ]:
# Ensure the source timestamp data is properly parsed to datetimes:
amz_forecast_df["timestamp"] = pd.to_datetime(amz_forecast_df["timestamp"])
# Create the month column:
amz_forecast_df["month"] = amz_forecast_df["timestamp"].dt.strftime("%Y-%m")

# Aggregate the data (sum):
amz_forecast_df = amz_forecast_df.groupby(
    ["month"] + actuals_dimensions
).agg(
    {
        "mean": "sum",
        "p10": "sum",
        "p50": "sum",
        "p90": "sum",
    }
)

# Force any negative (month-aggregated) predictions up to 0 sales:
amz_forecast_df[amz_forecast_df < 0] = 0

# Preview resulting dataframe:
amz_forecast_df

This forecast now matches our standard format, so we can add it to the evaluation set:

In [ ]:
FORECASTS["Amazon Forecast"] = NormalizedForecast(
    name="Amazon Forecast",
    df=amz_forecast_df,
    quantiles=["mean", "p10", "p50", "p90"],
)

## Estimating downstream costs of forecasting errors<a class="anchor" id="costs"></a>

Now we have a retrospective period selected, and our actual sales data as well as multiple candidate forecasts for the period - loaded and normalized.

From here we could of course calculate and compare basic accuracy-oriented metrics like RMSE, MAPE, or MASE to quantify which forecast was "closest" to actual recorded sales. Standard metrics like these are great for giving us a comparable view of the accuracy of different forecasts, but do little to help us answer the bigger question: **What's the value to our business** of considering a switch from forecast A to forecast B?

We come to an important but challenging insight:

> *The forecast itself has no value at all: Only the **business decisions it drives***

As forecasting analysts, this presents a challenge: Often these decisions are made by **humans** on **separate teams** - for example store- or category-managers, or sales reps. How can we quantify what we don't directly control?

Luckily, a second and equally important idea comes to our rescue:

> *It's okay for a business case to **start rough**, so long as it's **unbiased***

We're trying to estimate return on project investments here, not engineer components for the next space shuttle. Uncertainty and approximations are expected in business, and it should be acceptable to start simple and iteratively refine our model.

What *is* important is that we stay mindful of how approximate the estimate is, and try to avoid choosing assumptions that bias it excessively one way or the other (being ultra-conservative or overly-optimistic).

In the following sections we'll present some **basic, early-stage models** for estimating different business costs incurred due to forecasting errors. We aim to keep assumptions pretty high-level, so these metrics are applicable to many businesses.

Ultimately, **it's up to you** to refine and improve these estimates based on your specific business context and the data available: We'll talk more about these opportunities in each section.

### Cost of excess inventory<a class="anchor" id="inventory"></a>

Regardless of whether stock ordering or production planning processes are manual or automated, over-forecasting demand generally leads to over-ordering or over-producing stock.

While over-ordering has an immediate impact on free cash flow, the bottom-line cost can of course be complex to estimate:

- Can the excess stock be stored and sold in future periods? Or does it have limited shelf life that might make it a write-off?
- Will demand in future periods be sufficient to sell off the excess in reasonable time? (For example: seasonal goods or one-off crazes)
- What costs does storage incur? (For example: warehouse space, extra transportation, refrigeration...)
- What rate does stored inventory get lost to shrinkage? (For example: theft, accidental damage)
- If goods do expire, are sales truly First-In-First-Out? (For example: customers choosing milk cartons with longer expiry dates in-store)

We also need to make some assumptions about the **ordering/production decision process** to compare multiple forecasts: For example if human store managers ultimately make stocking decisions based on the demand forecast, it's not straightforward to say "what could have been" if we'd presented a different demand forecast at the time.

#### Starting simple

For a rough initial estimate, we'll compare our forecasts using the following assumptions:

- The business orders/produces exactly as many of each item as demand is forecast
- In each period (month) where the forecast exceeded actual demand, we multiply the excess by some per-item procurement cost
- The per-item cost can vary by period

You can think of the per-item cost as the full production/procurement cost of an item, if you're modelling the impact of over-ordering on free cash flow each period... Or some discounted per-period cost for storage and shrinkage, if your products have a longer shelf-life and you're interested in actual bottom-line losses.

First, we'll need the per-item costs:

In [ ]:
costs_df = util.analytics.filter_to_period(
    pd.read_parquet("s3://measuring-forecast-benefits-assets/dataset/v1/unit_costs.parquet"),
    period_start=PERIOD_START,
    period_end=PERIOD_END,
    timestamp_col_name="date",
)

costs_df.rename(columns={"date": "month"}, inplace=True)
costs_df

These costs are already aggregated at the month level, but the dimensions are coarser than our actual forecasts: By country instead of store location, and product type instead of individual SKU.

luckily it's fairly straightforward for us to map from location to country and SKU to product in the sample dataset, because the location and SKU IDs are just combinations of multiple fields.

In [ ]:
costs_df = costs_df.merge(
    loc_item_combos,
    on=["country", "product"],
    how="left",
).set_index(["month", "location", "item_id"])

costs_df

With per-item costs normalized, we're ready to join together our actuals and forecasts and estimate the costs of over-stocking.

In [ ]:
def join_forecasts(
    actuals: pd.DataFrame,
    forecasts: Dict[str, NormalizedForecast],
    actuals_zero_fill_cols: Optional[List[str]] = None,
):
    """Join actuals and (multiple) forecasts with multi-level column names"""
    result = actuals.copy()
    result.columns = pd.MultiIndex.from_arrays(
        [["Actual"] * len(actuals.columns), [c for c in actuals.columns]],
        names=["source", "column"],
    )
    for forecast in forecasts.values():
        if forecast.name == "Actual":
            raise ValueError(
                "'Actual' is reserved: You can't use this for a NormalizedForecast.name!"
            )
        forecast_norm = forecast.df.copy()
        forecast_norm.columns = pd.MultiIndex.from_arrays(
            [[forecast.name] * len(forecast_norm.columns), [c for c in forecast_norm.columns]],
            names=["source", "column"],
        )
        result = result.join(forecast_norm, how="outer")

    if actuals_zero_fill_cols:
        for colname in actuals_zero_fill_cols:
            result[("Actual", colname)] = result[("Actual", colname)].fillna(0)

    return result

In [ ]:
# Join the actuals and forecasts together:
# (Any missing actual sales records implies 0 sales for that item)
overstock_df = join_forecasts(actuals_df, FORECASTS, actuals_zero_fill_cols=["sales"])

# Add in the item costs data:
costs_tmp = costs_df.copy()
costs_tmp.columns = pd.MultiIndex.from_arrays(
    [["Unit Costs"] * len(costs_tmp.columns), [c for c in costs_tmp.columns]],
    names=["source", "column"],
)
overstock_df = overstock_df.join(costs_tmp, how="left")
del costs_tmp

# Any gaps in sales data should be interpreted as zero sales for that particular product/period/etc:
overstock_df.loc[:, ("Actual", "sales")].fillna(0, inplace=True)

# For each forecast, for each quantile, estimate the over-stock losses:
for forecast in FORECASTS.values():
    for quantile in forecast.quantiles:
        print(f"{forecast.name} - {quantile}")
        # Calculate how many units over real sales were forecast:
        overstock_df.loc[:, (forecast.name, f"{quantile}_overstock")] = (
            overstock_df[forecast.name][quantile] - overstock_df["Actual"]["sales"]
        ).clip(lower=0)
        # Multiply by item cost for the total loss:
        overstock_df.loc[:, (forecast.name, f"{quantile}_overstock_cost")] = (
            overstock_df[forecast.name][f"{quantile}_overstock"]
            * overstock_df["Unit Costs"]["unit_cost"]
        )

print("\n-- Missing values after join: --")
print(overstock_df.isna().sum(), "\n")
overstock_df

You might observe (hopefully very few) missing forecast values in the above table, but shouldn't have any gaps in actuals or item unit costs. This is because different models might have different criteria for *when* they can forecast: For example, the moving average may require some warm-up months, and Amazon Forecast may exclude items from the backtest export if their sales data starts very late (such as after the backtest window begins).

We can summarize and slice these detailed calculations as needed, to analyze the performance of each model.

For these summaries, we'll `dropna()` to ignore any records where not all models were able to forecast. This gives fairer comparisons (otherwise models with broader support would be penalized by having potentially-non-zero `overstock` costs where other models missing data), but doesn't quantify the value of one model having broader item support than another.

In [ ]:
# Filter to just the final costs columns:
overstock_costs = overstock_df[
    [
        (forecast.name, f"{quantile}_overstock_cost")
        for forecast in FORECASTS.values()
        for quantile in forecast.quantiles
    ]
].dropna()

# Top-level summary for each forecast and quantile:
overstock_costs.sum().map("${:,.2f}".format)

As a quick validation check here, it should make sense that the top-level `p90_overstock_cost` is much greater than the `p10_overstock_cost`: As discussed further [here](https://aws.amazon.com/blogs/machine-learning/amazon-forecast-now-supports-the-generation-of-forecasts-at-a-quantile-of-your-choice/), Amazon Forecast can generate quantile forecasts from p1 to p99 to characterize likely lower- and upper-bounds of actual demand, and reflect the uncertainty of demand as it changes over time.

If we take a very **low** quantile forecast, and only order/produce that many products, then of course we'll see minimal costs incurred due to over-stocking: The trade-off would be that we **miss out on a lot of potential sales** due to not having sufficient stock on hand. Those lost revenues are what we'll discuss in the next section.

Between the low-level and top-level views, you can of course also slice and summarize intermediate views for individual managers: For example by store as shown below.

In [ ]:
overstock_costs.groupby(["location"]).sum().applymap("${:,.2f}".format)

### Cost of stock-out events<a class="anchor" id="stockouts"></a>

On the opposite side to over-stocking, what happens when we *under-forecast* and don't order or manufacture enough product to meet customer demand?

Running out of stock of products is sometimes called a "stock-out" event, and this too is bad for business:

1. As an immediate result, the business will **lose revenue** for any customers who tried to buy the product but couldn't.
2. In the longer term, limited selection or patchy availability will **erode customer loyalty** and may harm the business' market share.

Of these factors, direct lost revenue (1) is perhaps the easier to start estimating. However, there are still potential complexities to be aware of:

- We need some kind of estimate of how many potential sales were missed on days/periods where stock ran out. Of course the original demand forecast itself is something we can use here, but:
    - In the typical situation where we're *comparing multiple forecasts*, which one should we take as our "best guess"?
    - In retrospect we do actually have more information available to us than when the forecast was originally made. For example if sales tracked above forecast for the first 20 days of the month before stock ran out, doesn't that mean our "best guess" of demand in the final 10 days would likely also be higher than the original forecast? Is it worth training a new model specifically to answer this? Or (if working with probabilistic models) choosing a higher quantile for the "best guess"?
- Sometimes actual sales can be restricted *even in low-stock periods* before inventory systems record stock finally dropping to zero:
    - In high-demand periods in physical retail, there may be periods where no stock is available on shelves despite inventory being available in back-room/storage
    - Unless stock counts and reconciliations are performed regularly, shrinkage factors like theft and loss can cause stock tracking systems to show limited availability when there is none in practice.

#### Starting simple

For a rough initial estimate, we'll compare our forecasts using the following assumptions:

1. The business orders/produces exactly as many of each item as demand is forecast
2. In each period (month) where the actual demand exceeded the forecast, we multiply the shortfall by the current item sale price
3. Item sale prices can vary over periods

Of course in practice, if (1) was true then (2) could never happen: Observed sales would be strictly less than or equal to the forecast in each month. For a self-consistent, online evaluation of one forecast model already in production, you could directly use inventory data to identify stock-out days and use the forecasted demand for those days as your basis. However, what you're really evaluating there is the *end-to-end loop* of what inventory decisions that forecast drove - so that doesn't really generalize to comparing multiple models. This is discussed further in the "[A/B Testing](#abtesting)" section below.

With this method you could choose to use either actual item sale prices (to model lost revenue) or just item margins (to model lost profit). We'll refer to "prices" for consistency, and start by loading that dataset:

In [ ]:
prices_df = util.analytics.filter_to_period(
    pd.read_parquet("s3://measuring-forecast-benefits-assets/dataset/v1/prices_promos.parquet"),
    period_start=PERIOD_START,
    period_end=PERIOD_END,
    timestamp_col_name="date",
)
prices_df

In this example price data and actual sales are both available at the daily level which might allow for a more detailed model of when in the month stock would run out and what the real average price of missed sales might be. However, since our baseline forecast is only monthly with no breakdown by day, such detailed comparison would require some assumptions anyway.

For an basic view, we'll just take an average unit price for the month over all days (without weighting by actual or forecast sales on those days):

In [ ]:
prices_df["month"] = prices_df["date"].dt.strftime("%Y-%m")

prices_df = prices_df.groupby(
    ["month", "country", "product"]
).agg(
    {"unit_price": "mean"}
).reset_index().merge(
    loc_item_combos,
    on=["country", "product"],
    how="left",
).set_index(["month", "location", "item_id"])

prices_df

As for over-stock cost estimation earlier, we're now ready to join our actual data with forecasts and item prices, to estimate the lost revenues due to stock-out events:

In [ ]:
# Join the actuals and forecasts together:
# (Any missing actual sales records implies 0 sales for that item)
stockout_df = join_forecasts(actuals_df, FORECASTS, actuals_zero_fill_cols=["sales"])

# Add in the item costs data:
prices_tmp = prices_df.copy()
prices_tmp.columns = pd.MultiIndex.from_arrays(
    [["Unit Prices"] * len(prices_tmp.columns), [c for c in prices_tmp.columns]],
    names=["source", "column"],
)
stockout_df = stockout_df.join(prices_tmp, how="left")
del prices_tmp

# Any gaps in sales data should be interpreted as zero sales for that particular product/period/etc:
stockout_df.loc[:, ("Actual", "sales")].fillna(0, inplace=True)

# For each forecast, for each quantile, estimate the over-stock losses:
for forecast in FORECASTS.values():
    for quantile in forecast.quantiles:
        print(f"{forecast.name} - {quantile}")
        # Calculate how many units over real sales were forecast:
        stockout_df.loc[:, (forecast.name, f"{quantile}_missedsales")] = (
            stockout_df["Actual"]["sales"] - stockout_df[forecast.name][quantile]
        ).clip(lower=0)
        # Multiply by item cost for the total loss:
        stockout_df.loc[:, (forecast.name, f"{quantile}_missedsales_rev")] = (
            stockout_df[forecast.name][f"{quantile}_missedsales"]
            * stockout_df["Unit Prices"]["unit_price"]
        )

print("\n-- Missing values after join: --")
print(stockout_df.isna().sum(), "\n")
stockout_df

As before, there may be a small number of missing forecast values due to differing criteria for each model to be able to forecast.

Again, we can drop any records with missing data and aggregate this detail into a global summary for each forecast and quantile:

In [ ]:
# Filter to just the final costs columns:
stockout_costs = stockout_df[
    [
        (forecast.name, f"{quantile}_missedsales_rev")
        for forecast in FORECASTS.values()
        for quantile in forecast.quantiles
    ]
].dropna()

# Top-level summary for each forecast and quantile:
stockout_costs.sum().map("${:,.2f}".format)

The results here are somewhat opposite to the over-stocking costs calculated earlier: You'll see losses are much greater for *low-quantile* forecasts (like `p10` from Amazon Forecast or SageMaker Canvas) than high-quantile forecasts like `p90`. If we take a lower-bound forecast and only order sufficient stock to cover that, then of course we would expect bigger losses in potential revenue due to running out of inventory.

As in the previous section, we could also summarize this to different levels for example by individual store:

In [ ]:
stockout_costs.groupby(["location"]).sum().applymap("${:,.2f}".format)

### Total costs and benefits<a class="anchor" id="totalcosts"></a>

So far we've identified and estimated multiple business inefficiencies caused by forecasting errors:

- Reduced free cash flow or bottom-line written-off cost of excess inventory due to over-ordering (depending whether you used full item procurement cost, or just write-off cost proportion)
- Lost revenue or bottom-line profit from sales missed due to running out of stock (depending whether you used full item sale price, or just margin)

Many businesses will apply **different weight to these different metrics**, and so it may not be appropriate to simply sum up the dollar values: Different trade-offs might be important to a business between revenue maximization and cost reduction.

One way to combine all the factors would be to express each in terms of bottom-line impact. Another could be to take a weighted combination of revenue growth and cost reduction. However you tackle it, you might see **trade-offs** like we showed in the extreme case of choosing biased upper-bound or lower-bound forecasts!

The code below shows a way you might bring the top-level summary costs together for the different forecast models. You should be able to see that Amazon Forecast / SageMaker Canvas quantiles like `mean` or `p50` significantly out-perform the Moving Average baseline forecast overall, but with different trade-offs between the business metrics:

In [ ]:
# Create MultiIndex series by [forecast][metric]
tmp = pd.concat([stockout_costs.sum(), overstock_costs.sum()]).rename("value").reset_index(level=1)

# Split the raw metric names (e.g. mean_missedsales_rev) to their quantile and metric:
column_parted = tmp["column"].str.partition("_")  # Assume no underscores in quantile names!
tmp["quantile"] = column_parted[0]
tmp["metric"] = column_parted[2]
tmp.drop(columns=["column"], inplace=True)

# Index and pivot the data for a nice view:
summary = tmp.set_index(["quantile"], append=True).pivot(columns=["metric"])

summary.applymap("${:,.2f}".format)

In [ ]:
ax = (summary / 1000).plot.barh(
    figsize=(10, 4),  # (width, height)
    stacked=True,
    title="Business Inefficiencies by Forecast (Lower is Better)",
)
ax.grid(axis="x")
ax.set_xlabel("Thousand Dollars")
ax.set_ylabel("Forecast Model, Quantile")
ax.get_figure().savefig("dataset/result-summary.png", bbox_inches="tight")

In our tests as shown below (your exact numbers may vary), re-stocking based on Amazon Forecast `mean` or `p50` (median) quantiles delivered the best performance with combined business inefficiencies around &#36;1M. Even taking an extreme quantile from Forecast like `p10` or `p90` delivered better results than ordering based on the recent moving average of sales (around &#36;7M inefficiencies). SageMaker Canvas performed similarly to Forecast, which is no great surprise as it uses Amazon Forecast under the hood for forecasting models. As discussed in the Canvas notebook, our data preparation was a little different between the two AI/ML services and likely biased the comparison somewhat against Canvas.

![](img/final-summary.png "Bar chart and data table showing overall lost revenue and increased cost for each forecasting strategy")

## Online evaluation and A/B testing<a class="anchor" id="abtesting"></a>

As the examples above have hopefully started to illustrate, when we talk about the "business value of better demand forecasting", what we *really* mean is the value of the **decisions you make based on the forecast**.

Our example metrics have both been about the stock ordering / production planning process, making some **assumptions** about how your ordering or production choices might have changed under different forecasts, so we can model whether you would've had too much or too little stock on hand to meet demand.

For many businesses these decisions aren't (yet?) fully automated, so you might well question those assumptions: How do I really know what stock the store manager would have ordered if I showed them a forecast for X instead of Y last month?

Here are two ways you could refine your estimates further in situations like these:

1. **Extend your retrospective model** to try and simulate the complexities of what would happen to ordering decisions and therefore stock levels under different forecasts.
    - As we said earlier, a business case doesn't have to be exact: If you think there are important dependencies or feedbacks not being captured, you can iteratively refine your model starting from something simple.
2. **A/B test in the real-world** to try and directly measure your impact.

With manual processes, simulating "what could have been" may be hard but estimating the cost of your **actual** waste (stock left in inventory, or forecasted sales during stock-out periods) should still be practical.

You could consider running limited live pilots where both the old and new proposed forecasting models are used *in parallel*, to try and quantify real-world rewards.

Applying this at a fine grain (for example randomly selecting products to use forecast model A or B, instead of big-bang deployment to an entire store or product category); and keeping the model selection hidden from decision-makers; are two practices you could apply to separate real signals from noise and avoid bias driven by big, widely-communicated changes.

## Conclusions<a class="anchor" id="conclusions"></a>

When comparing and refining forecasting models, it's important to **elevate your analysis** from science-oriented accuracy metrics to business-oriented value metrics where possible - to help you understand the real-world impact of model improvements, when to dive deeper, and when to shift focus to other more urgent projects.

It's common for the downstream impacts of forecasting improvement to be complex, and for human decision-makers to intervene between the initial forecast output and the final outcomes that drive business costs or revenue.

This doesn't mean forecast owning teams should give up trying to understand the impact of their investments though: You can start out with simple heuristics, and iteratively refine as you explore the business context. If counter-factual "what could have been" analysis is too complex in your case for estimates to be useful, you could explore running live A/B tests to track the difference in real-world value between your candidate models.

Inventory management is one practical place to start, and in this example we showed some simplistic models for estimating how demand forecasting errors might contribute to increased costs, impaired cashflow, and missed revenue through the stock ordering/production decisions they drive.

By progressively building maturity in modelling the end-to-end impacts of forecasting, you can start to unlock other forecasting-related use-cases like:

- Pricing optimization, by analyzing the price elasticity of the demand forecast
- Increased decision automation, by building confidence in automated ordering rules over time
- End-to-end supply chain optimization, starting to consider lead times and other factors

For more information about how AWS can help you build a data-driven supply chain and incorporate ML into your business, check out:

- [AWS Supply Chain](https://aws.amazon.com/aws-supply-chain/), a fully-managed service to unify supply chain data and provide actionable, ML-powered insights
- [AWS Supply Chain Competency Partners](https://aws.amazon.com/industrial/supply-chain-management/partners/), for AWS partners with validated experience in supply chain solutions
- ...And if you'd like to dive deeper, the [Operations Research](https://www.amazon.science/research-areas/operations-research-and-optimization) and [Machine Learning](https://www.amazon.science/research-areas/machine-learning) sections of the [Amazon Science blog](https://www.amazon.science/) share recent research from Amazon on related topics.